![alt text](./pageheader_rose2_babies.jpg)

# Analysis and visualisation of the ventilator parameters obtained with 1 Hz sampling rate in an individual recording

#### Author: Dr Gusztav Belteki


This notebook imports the "slow" (1 Hz) ventilator data of selected recordings. 

This Notebook also generates Figure 3 as presented in the paper describing _Ventiliser_: David Chong, Colin J Morley & Gusztav Belteki: __Computational analysis of neonatal ventilator waveforms and loop__. _Pediatric Research_, in press. 

The _Ventiliser_ Python package presented in the paper can be downloaded from [here](https://github.com/barrinalo/Ventiliser).

With questions and for more information please contact us: gbelteki@aol.com (Gusztav Belteki), dtwc3@cam.ac.uk (David Chong)

### 1. Import the required modules and set options

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

import matplotlib.pyplot as plt
from matplotlib import dates

import os
import datetime

# Module containing function to import and process data
import slow_data_analyser

In [ ]:
# Select the recording to analyse
recording = 'DG007'

### 2. Import and pre-process data 

In [ ]:
%%time

clinical_details = slow_data_analyser.process_clinical_data(recording)
pCO2s = slow_data_analyser.process_blood_gases(recording)
vent_modes = slow_data_analyser.process_vent_modes(recording)
vent_settings = slow_data_analyser.process_vent_settings(recording)
slow_measurements = slow_data_analyser.process_slow_measurements(recording)

### 3. Exploratory analysis on the data

#### Calculate recording duration

In [ ]:
recording_duration = {}

for part in slow_measurements:
    recording_duration[part] = slow_measurements[part].index[-1] - slow_measurements[part].index[0]


for part, duration in recording_duration.items():
    print('part %d' % part, ' ' * 10,  str(duration))

#### What is the number and proportion of `na` values in each recording

In [ ]:
# Absolute number or missing values

for part in slow_measurements:
    print('part %d' % part, ' ' * 10, 
          'proportion of missing values: %.5f %%' % (slow_measurements[part].isna().sum().sum() * 100 / \
    (slow_measurements[part].shape[0] * slow_measurements[part].shape[1]),))
            

### 4. Basic clinical data

In [ ]:
print('Recording:             %s\n' % recording)
print('Recording period:      %s' % clinical_details.loc[recording]['Recording period'])
print('Gestation:             %d weeks' % clinical_details.loc[recording]['Gestation'])
print('Corrected gestation:   %d weeks' % clinical_details.loc[recording]['Corrected gestation'])
print('Birth weight:          %d grams' % clinical_details.loc[recording]['Birth weight'])
print('Current weight:        %d grams' % clinical_details.loc[recording]['Current weight'])
print('Diagnoses:             %s' % clinical_details.loc[recording]['Main diagnoses'])
print('Comments:              %s' % clinical_details.loc[recording]['Comment'])

### 5. Generate 1-minute means / medians and 3-hour rolling means / medians as appropriate

In [ ]:
%%time

slow_measurements_1min_mean, slow_measurements_1min_median, \
slow_measurements_3h_rolling_mean, slow_measurements_3h_rolling_median = \
slow_data_analyser.aggregator(slow_measurements)

### 6. Combine selected columns of the Dataframes into one

In [ ]:
%%time

slow_measurements_comb = slow_data_analyser.frame_combiner(slow_measurements)
slow_measurements_comb_1min_mean = slow_data_analyser.frame_combiner(slow_measurements_1min_mean)
slow_measurements_comb_1min_median = slow_data_analyser.frame_combiner(slow_measurements_1min_median)
slow_measurements_comb_3h_rolling_mean = slow_data_analyser.frame_combiner(slow_measurements_3h_rolling_mean)
slow_measurements_comb_3h_rolling_median = slow_data_analyser.frame_combiner(slow_measurements_3h_rolling_median)

### 7. Identify a sub-period

In [ ]:
# Start and End of recording

str(slow_measurements_comb.index[0]), str(slow_measurements_comb.index[-1])

In [ ]:
start = '' or slow_measurements_comb.index[0]
end = '' or slow_measurements_comb.index[-1]

if len(slow_measurements_comb) > len(slow_measurements_comb[start:end]):

    tag = ('_' + str(start)[8:10]+ str(start)[5:7] + str(start)[2:4] + '_' + str(start)[11:13] 
      + str(start)[14:16] + '_' + str(end)[8:10]+ str(end)[5:7] + str(end)[2:4] + 
     '_' + str(end)[11:13] + str(end)[14:16] + '_')

else:
    tag = '_'
    
tag

In [ ]:
len(slow_measurements_comb), len(slow_measurements_comb[start:end])

### 8. Descriptive statistics (on the selected period)

In [ ]:
%%time

percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]
stats = {}

stats['1sec_data'] = round(slow_measurements_comb[start:end].describe(percentiles = percentiles), 4)
stats['1sec_data'].dropna(subset = ['mean'], axis = 1, inplace = True)
stats['1min_mean'] = round(slow_measurements_comb_1min_mean[start:end].describe(percentiles = percentiles), 4)
stats['1min_mean'].dropna(subset = ['mean'], axis = 1, inplace = True)
stats['1min_median'] = round(slow_measurements_comb_1min_median[start:end].describe(percentiles = percentiles), 4)
stats['1min_median'].dropna(subset = ['mean'], axis = 1, inplace = True)
stats['3h_rolling_mean'] = round(slow_measurements_comb_3h_rolling_mean[start:end].describe(percentiles = 
    percentiles), 4)
stats['3h_rolling_mean'].dropna(subset = ['mean'], axis = 1, inplace = True)
stats['3h_rolling_median'] = round(slow_measurements_comb_3h_rolling_median[start:end].describe(percentiles = 
    percentiles), 4)
stats['3h_rolling_median'].dropna(subset = ['mean'], axis = 1, inplace = True)

In [ ]:
stats['1sec_data']

In [ ]:
stats['1min_median']

In [ ]:
stats['3h_rolling_mean']

In [ ]:
stats['3h_rolling_median']

### 9. Basic information on ventilation

In [ ]:
vent_modes_used = []

for item in vent_modes.values():
    lst = item['Text'].values
    lst = [item.split('-')[1] for item in lst if item.startswith(' Mode')]
    vent_modes_used.extend(lst)
    
vent_modes_used = sorted(set(vent_modes_used))

In [ ]:
print('Recording:                       %s\n' % recording)
print('Recording start:                 %s' % str(slow_measurements_comb.index[0]))
print('Recording end:                   %s' % str(slow_measurements_comb.index[-1]))
print('Recording duration:              %s' % recording_duration)
print('Ventilation modes:              ', *sorted(set(vent_modes_used)))

### 10. Save Clinical and ventilation information and statistics in files

In [ ]:
# Images and reports will be written on an external USB stick
PATH = ['/Volumes', 'guszti', 'data_dump', 'draeger', 'analysis_individual']

if not os.path.isdir(os.path.join(*PATH)):
    os.makedirs(os.path.join(*PATH))
DATA_DUMP = os.path.join(*PATH)

In [ ]:
if not os.path.isdir(os.path.join(DATA_DUMP, recording)):
    os.makedirs(os.path.join(DATA_DUMP, recording))

#### Basic clinical data

In [ ]:
fileout = open('%s/%s/%s_%s.%s' % (DATA_DUMP, recording, recording, 'clinical_info', 'txt'), 'w')
fileout.write('Recording:             %s\n\n' % recording)
fileout.write('Recording period:      %s\n' % clinical_details.loc[recording]['Recording period'])
fileout.write('Gestation:             %d weeks\n' % clinical_details.loc[recording]['Gestation'])
fileout.write('Corrected gestation:   %d weeks\n' % clinical_details.loc[recording]['Corrected gestation'])
fileout.write('Birth weight:          %d grams\n' % clinical_details.loc[recording]['Birth weight'])
fileout.write('Current weight:        %d grams\n' % clinical_details.loc[recording]['Current weight'])
fileout.write('Diagnoses:             %s\n' % clinical_details.loc[recording]['Main diagnoses'])
fileout.write('Comments:              %s\n' % clinical_details.loc[recording]['Comment'])
fileout.close()

#### Basic information on ventilation

In [ ]:
fileout = open('%s/%s/%s_%s.%s' % (DATA_DUMP, recording, recording, 'vent_info', 'txt'), 'w')

fileout.write('Recording:                       %s\n\n' % recording)
fileout.write('Recording start:                 %s\n' % str(slow_measurements_comb.index[0]))
fileout.write('Recording end:                   %s\n' % str(slow_measurements_comb.index[-1]))
fileout.write('Recording duration:              %s\n' % recording_duration)
fileout.write('Ventilation modes:               ') 
print( *set(vent_modes_used), sep = ', ', file = fileout)
    
fileout.close()

#### Save ventilation modes and settings as Excel files

In [ ]:
writer = pd.ExcelWriter(os.path.join(DATA_DUMP, recording, '%s_%s') % 
                        (recording, 'ventilation_modes_settings.xlsx'))

for part in vent_modes:
    vent_modes[part].to_excel(writer, 'vent_modes_part_%s' % part)
    vent_settings[part].to_excel(writer,'vent_settings_%s' % part)

writer.save()

#### Save statistics on the selected period

In [ ]:
writer = pd.ExcelWriter(os.path.join(DATA_DUMP, recording, '%s%s%s' % (recording, tag, 'stats.xlsx')))
for item in stats:
    stats[item].to_excel(writer, item)
writer.save()

## 11. Plots

In [ ]:
# Start and end of recording

str(slow_measurements_comb[start:end].index[0]), str(slow_measurements_comb[start:end].index[-1])

In [ ]:
segment_1_start = '2015-10-19 22:00:00' 
segment_1_end   = '2015-10-19 23:00:00'

In [ ]:
segment_2_start = '2015-10-20 17:00:00' 
segment_2_end   = '2015-10-20 18:00:00'

### `RR` and `RRset`

In [ ]:
if {'RR', 'RRset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['RR'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb[start:end]['RRset'].plot(ax = ax, color = 'black', linewidth = 2, x_compat = True)
    
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 105, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 105, '2', fontsize = 14)
    
    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 150)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('Breaths/min', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['RR', 'RRset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'RR_RRset', filetype)),
        dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'RR', 'RRset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['RR'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb[start:end]['RRset'].plot(ax = ax, color = 'black', linewidth = 2, 
                                                          x_compat = True)
    
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 105, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 105, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 150)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('Breaths/min', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['RR', 'RRset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'RR_RRset_1min_mean', filetype)),
        dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'RR', 'RRset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['RR'].plot(ax = ax, color = 'blue', 
                                                          x_compat = True )
    slow_measurements_comb[start:end]['RRset'].plot(ax = ax, color = 'black', linewidth = 2, 
                                                          x_compat = True)
    
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 105, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 105, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 150)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('Breaths/min', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['RR', 'RRset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'RR_RRset_3h_rolling_mean',
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `RR`, `RRmand`, `RRspon`,  `RRset`

In [ ]:
if {'RR', 'RRmand', 'RRspon', 'RRset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['RR'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb[start:end]['RRmand'].plot(ax = ax, color = 'red', x_compat = True )
    slow_measurements_comb[start:end]['RRspon'].plot(ax = ax, color = 'green', x_compat = True )
    slow_measurements_comb[start:end]['RRset'].plot(ax = ax, color = 'black', linewidth = 2, x_compat = True)
    
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 105, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 105, '2', fontsize = 14)
    
    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 150)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('Breaths/min', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['RR', 'RRmand', 'RRspon', 'RRset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'RRmand_spon_set', filetype)),
        dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'RR', 'RRmand', 'RRspon', 'RRset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['RR'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb_1min_mean[start:end]['RRmand'].plot(ax = ax, color = 'red', x_compat = True )
    slow_measurements_comb_1min_mean[start:end]['RRspon'].plot(ax = ax, color = 'green', x_compat = True )
    slow_measurements_comb[start:end]['RRset'].plot(ax = ax, color = 'black', linewidth = 2, x_compat = True)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 105, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 105, '2', fontsize = 14)
    
    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 150)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('Breaths/min', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['RR', 'RRmand', 'RRspon', 'RRset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'RRmand_spon_set_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'RR', 'RRmand', 'RRspon', 'RRset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['RR'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb_3h_rolling_mean[start:end]['RRmand'].plot(ax = ax, color = 'red', x_compat = True )
    slow_measurements_comb_3h_rolling_mean[start:end]['RRspon'].plot(ax = ax, color = 'green', x_compat = True )
    slow_measurements_comb[start:end]['RRset'].plot(ax = ax, color = 'black', linewidth = 2, x_compat = True)
    
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 105, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 105, '2', fontsize = 14)
    
    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 150)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('Breaths/min', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['RR', 'RRmand', 'RRspon', 'RRset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'RRmand_spon_set_3h_rolling_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

### `PIP`, `MAP`, `PEEP`,  `Pmax`

(VG ventilation)

In [ ]:
if {'PIP', 'MAP', 'PEEP', 'Pmax'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['PIP'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['MAP'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb[start:end]['PEEP'].plot(ax = ax, color = 'green', x_compat = True )
    slow_measurements_comb[start:end]['Pmax'].plot(ax = ax, color = 'black', linewidth = 2, 
                                                            x_compat = True)
    
   # If you want to highlight sub-periods on the graphs, uncomment these lines and 
    # insert date and time in the correct format
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)
    
    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 50)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['PIP', 'MAP', 'PEEP', 'Pmax'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'PIP_MAP_PEEP_Pmax', filetype)),
        dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'PIP', 'MAP', 'PEEP', 'Pmax'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['PIP'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb_1min_mean[start:end]['MAP'].plot(ax = ax, color = 'red', x_compat = True )
    slow_measurements_comb_1min_mean[start:end]['PEEP'].plot(ax = ax, color = 'green', x_compat = True )
    slow_measurements_comb[start:end]['Pmax'].plot(ax = ax, color = 'black', linewidth = 2, x_compat = True)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)
    
    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 50)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['PIP', 'MAP', 'PEEP', 'Pmax'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
            'PIP_MAP_PEEP_Pmax_1min_mean', filetype)), dpi = dpi, format = filetype, 
                bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'PIP', 'MAP', 'PEEP', 'Pmax'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['PIP'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb_3h_rolling_mean[start:end]['MAP'].plot(ax = ax, color = 'red', x_compat = True )
    slow_measurements_comb_3h_rolling_mean[start:end]['PEEP'].plot(ax = ax, color = 'green', x_compat = True )
    slow_measurements_comb[start:end]['Pmax'].plot(ax = ax, color = 'black', linewidth = 2, x_compat = True)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 50)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['PIP', 'MAP', 'PEEP', 'Pmax'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'PIP_MAP_PEEP_Pmax_3h_rolling_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

### `PIP`, `MAP`, `PEEP`,  `PIP_set`

(non-VG ventilation)

In [ ]:
if {'PIP', 'MAP', 'PEEP', 'PIP_set'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['PIP'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['MAP'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb[start:end]['PEEP'].plot(ax = ax, color = 'green', x_compat = True )
    slow_measurements_comb[start:end]['PIP_set'].plot(ax = ax, color = 'black', linewidth = 2, 
                                                            x_compat = True)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)
        
    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 50)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['PIP', 'MAP', 'PEEP', 'PIP_set'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'PIP_MAP_PEEP_PIPset', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'PIP', 'MAP', 'PEEP', 'PIP_set'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['PIP'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb_1min_mean[start:end]['MAP'].plot(ax = ax, color = 'red', x_compat = True )
    slow_measurements_comb_1min_mean[start:end]['PEEP'].plot(ax = ax, color = 'green', x_compat = True )
    slow_measurements_comb[start:end]['PIP_set'].plot(ax = ax, color = 'black', linewidth = 2, x_compat = True)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 50)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['PIP', 'MAP', 'PEEP', 'PIP_set'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
            'PIP_MAP_PEEP_PIPset_1min_mean', filetype)), dpi = dpi, format = filetype, 
                bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'PIP', 'MAP', 'PEEP', 'PIP_set'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['PIP'].plot(ax = ax, color = 'blue', x_compat = True )
    slow_measurements_comb_3h_rolling_mean[start:end]['MAP'].plot(ax = ax, color = 'red', x_compat = True )
    slow_measurements_comb_3h_rolling_mean[start:end]['PEEP'].plot(ax = ax, color = 'green', x_compat = True )
    slow_measurements_comb[start:end]['PIP_set'].plot(ax = ax, color = 'black', linewidth = 2, x_compat = True)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 50)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['PIP', 'MAP', 'PEEP', 'PIP_set'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'PIP_MAP_PEEP_PIP_set_3h_rolling_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

### `VTmand` and `VTset`
(VG ventilation)

In [ ]:
if {'VTmand', 'VTset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['VTmand'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb[start:end]['VTset'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 30)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VTmand_VTset', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VTmand', 'VTset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['VTmand'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb[start:end]['VTset'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 12)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VTmand_VTset_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VTmand', 'VTset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['VTmand'].plot(ax = ax, color = 'blue', 
                                                                     x_compat = True,)
    slow_measurements_comb[start:end]['VTset'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 12)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VTmand_VTset_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `VTimand`, `VTmand`, `VTemand` and `VTset`
(VG ventilation)


In [ ]:
if {'VTimand', 'VTmand', 'VTemand', 'VTset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['VTimand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['VTmand'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb[start:end]['VTemand'].plot(ax = ax, color = 'green', x_compat = True,)
    slow_measurements_comb[start:end]['VTset'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)
    
    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 30)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTimand', 'VTmand', 'VTemand', 'VTset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VTimand_VTmand_VTemand_VTset', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VTimand', 'VTmand', 'VTemand', 'VTset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['VTimand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_1min_mean[start:end]['VTmand'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb_1min_mean[start:end]['VTemand'].plot(ax = ax, color = 'green', x_compat = True,)
    slow_measurements_comb[start:end]['VTset'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 12)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTimand', 'VTmand', 'VTemand', 'VTset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'VTimand_VTmand_VTemand_VTset_1min_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VTimand', 'VTmand', 'VTemand', 'VTset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['VTimand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_3h_rolling_mean[start:end]['VTmand'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb_3h_rolling_mean[start:end]['VTemand'].plot(ax = ax, color = 'green', x_compat = True,)
    slow_measurements_comb[start:end]['VTset'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 12)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTimand', 'VTmand', 'VTemand', 'VTset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'VTimand_VTmand_VTemand_VTset_3h_rolling_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

### `VTimand`, `VTmand`, `VTemand` 
(non-VG ventilation)

In [ ]:
if {'VTimand', 'VTmand', 'VTemand',}.issubset(slow_measurements_comb.columns) and \
   'VTset' not in slow_measurements_comb.columns:

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['VTimand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['VTmand'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb[start:end]['VTemand'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 30)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTimand', 'VTmand', 'VTemand',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VTimand_VTmand_VTemand', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VTimand', 'VTmand', 'VTemand',}.issubset(slow_measurements_comb.columns) and \
   'VTset' not in slow_measurements_comb.columns:

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['VTimand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_1min_mean[start:end]['VTmand'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb_1min_mean[start:end]['VTemand'].plot(ax = ax, color = 'green', x_compat = True,)
    
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 12)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTimand', 'VTmand', 'VTemand',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'VTimand_VTmand_VTemand_1min_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VTimand', 'VTmand', 'VTemand',}.issubset(slow_measurements_comb.columns) and \
   'VTset' not in slow_measurements_comb.columns:
    
    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['VTimand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_3h_rolling_mean[start:end]['VTmand'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb_3h_rolling_mean[start:end]['VTemand'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 12)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTimand', 'VTmand', 'VTemand',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'VTimand_VTmand_VTemand_3h_rolling_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

### `VTmand`, `VTspon`, and `VTset`
(VG ventilation)

In [ ]:
if {'VTmand', 'VTspon', 'VTset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['VTmand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['VTspon'].plot(ax = ax, color = 'green', x_compat = True,)
    slow_measurements_comb[start:end]['VTset'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 30)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTspon', 'VTset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VTmand_VTspon_VTset', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VTmand', 'VTspon', 'VTset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['VTmand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_1min_mean[start:end]['VTspon'].plot(ax = ax, color = 'green', x_compat = True,)
    slow_measurements_comb[start:end]['VTset'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 12)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTspon', 'VTset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'VTmand_VTspon_VTset_1min_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VTmand', 'VTspon', 'VTset'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['VTmand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_3h_rolling_mean[start:end]['VTspon'].plot(ax = ax, color = 'green', x_compat = True,)
    slow_measurements_comb[start:end]['VTset'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 12)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTspon', 'VTset'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'VTmand_VTspon_VTset_3h_rolling_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

### `VTmand`, `VTspon`
(non-VG ventilation)

In [ ]:
if {'VTmand', 'VTspon',}.issubset(slow_measurements_comb.columns)and \
   'VTset' not in slow_measurements_comb.columns:

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['VTmand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['VTspon'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 20, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 20, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 30)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTspon',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VTmand_VTspon', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VTmand', 'VTspon',}.issubset(slow_measurements_comb.columns) and \
   'VTset' not in slow_measurements_comb.columns:

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['VTmand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_1min_mean[start:end]['VTspon'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 12)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTspon',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'VTmand_VTspon_1min_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VTmand', 'VTspon',}.issubset(slow_measurements_comb.columns) and \
   'VTset' not in slow_measurements_comb.columns:

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['VTmand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_3h_rolling_mean[start:end]['VTspon'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 12)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VTmand', 'VTspon',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'VTmand_VTspon_3h_rolling_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

###  `MV`

In [ ]:
if {'MV'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['MV'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

    # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MV'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MV', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MV',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['MV'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

    # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MV',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MV_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MV'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['MV'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

    # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MV',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MV_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `MVi`, `MV`, `MVe`

In [ ]:
if {'MVi', 'MV', 'MVe',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['MVi'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['MV'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb[start:end]['MVe'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

    # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVi', 'MV', 'MVe',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MVi_MV_MVe', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MVi', 'MV', 'MVe',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['MVi'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_1min_mean[start:end]['MV'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb_1min_mean[start:end]['MVe'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

    # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVi', 'MV', 'MVe',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MVi_MV_MVe_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MVi', 'MV', 'MVe',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['MVi'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_3h_rolling_mean[start:end]['MV'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb_3h_rolling_mean[start:end]['MVe'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

    # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVi', 'MV', 'MVe',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MVi_MV_MVe_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `MVemand`, `MVespon`

In [ ]:
if {'MVemand', 'MVespon',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['MVemand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['MVespon'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)
    
    # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVemand', 'MVespon',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MVemand_MVespon', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MVemand', 'MVespon',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['MVemand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_1min_mean[start:end]['MVespon'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

     # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVemand', 'MVespon',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'MVemand_MVespon_1min_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MVemand', 'MVespon',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['MVemand'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_3h_rolling_mean[start:end]['MVespon'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

     # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVemand', 'MVespon',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 
        'MVemand_MVespon_3h_rolling_mean', filetype)), dpi = dpi, format = filetype, 
            bbox_inches='tight', pad_inches=0.1,)

### `MVspon_pc`
Contribution of spontaneuous breaths to total minute ventilation (%)

In [ ]:
if {'MVspon_pc',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['MVspon_pc'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 80, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 80, '2', fontsize = 14)

    ax.set_ylim(0, 100)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('%', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVspon',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MVspon_pc', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MVspon_pc',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['MVspon_pc'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 80, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 80, '2', fontsize = 14)

    ax.set_ylim(0, 100)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('%', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVspon',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MVspon_pc_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MVspon_pc',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['MVspon_pc'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 80, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 80, '2', fontsize = 14)

    ax.set_ylim(0, 100)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('%', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVspon',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MVspon_pc_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `MVleak`
The minute volume escaping as leak around the ET tube or via a chest drain

In [ ]:
if {'MVleak',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['MVleak'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

     # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVleak',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MVleak', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MVleak',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_median[start:end]['MVleak'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

     # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVleak',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MVleak_1min_median', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MVleak',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_median[start:end]['MVleak'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.6, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.6, '2', fontsize = 14)

     # For HFOV, MV values are larger
    if 'DCO2' in slow_measurements_comb.columns:
        ax.set_ylim(0, 3)
        
    else:
        ax.set_ylim(0, 0.8)
        
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MVleak',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MVleak_3h_rolling_median', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `Leak`
Leak around the endotracheal tube or via a chest drain

In [ ]:
if {'Leak',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['Leak'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 80, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 80, '2', fontsize = 14)

    ax.set_ylim(0, 100)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('%', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Leak',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Leak', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'Leak',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_median[start:end]['Leak'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 80, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 80, '2', fontsize = 14)

    ax.set_ylim(0, 100)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('%', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Leak',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Leak_1min_median', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'Leak',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_median[start:end]['Leak'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 80, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 80, '2', fontsize = 14)

    ax.set_ylim(0, 100)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('%', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Leak',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Leak_3h_rolling_median', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `FiO2`

In [ ]:
if {'FiO2',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['FiO2'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 80, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 80, '2', fontsize = 14)
        
    ax.set_ylim(0, 100)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('%', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['FiO2',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'FiO2', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `Cdyn`
Dynamic compliance

In [ ]:
if {'Cdyn',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['Cdyn'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 12, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 12, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 15)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Cdyn',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Cdyn', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'Cdyn',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['Cdyn'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 2, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 2, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 10)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Cdyn',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Cdyn_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'Cdyn',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['Cdyn'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 2, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 2, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    ax.set_ylim(0, 5)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Cdyn',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Cdyn_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `Cdyn` and `C20/Cdyn`
Dynamic compliance and the compliance in the last 20% of inspiration

In [ ]:
if {'C20/Cdyn',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['Cdyn'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 20)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('ratio', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['C20/Cdyn',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'C20_Cdyn', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'C20/Cdyn',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['Cdyn'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 5)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('ratio', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['C20/Cdyn',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'C20_Cdyn_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'C20/Cdyn',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['Cdyn'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.8, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.8, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 5)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('ratio', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['C20/Cdyn',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'C20_Cdyn_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `R`
Airway resistance

In [ ]:
if {'R',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['R'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 100, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 100, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 1000)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar/L/s', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Resistance',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'R', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'R',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['R'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 100, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 100, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 1000)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar/L/s', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Resistance',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'R_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'R',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['R'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 100, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 100, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 500)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar/L/s', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Resistance',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'R_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `TC`
Time constant

In [ ]:
if {'TC',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['TC'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 1, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 1, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 5)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('seconds', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Time constant',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'TC', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'TC',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['TC'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.5, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.5, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 0.5)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('seconds', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Time constant',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'TC_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'TC',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['TC'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.25, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.25, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 0.5)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('seconds', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Time constant',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'TC_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### `TC` and `TCe`
Time constant and expiratory time constant

In [ ]:
if {'TC', 'TCe'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['TC'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_1min_mean[start:end]['TCe'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.5, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.5, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 1)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('seconds', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['TC', 'TCe'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'TC_TCe_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'TC', 'TCe'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['TC'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb_3h_rolling_mean[start:end]['TCe'].plot(ax = ax, color = 'green', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 0.25, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 0.25, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 0.7)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('s', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['TC', 'TCe'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'TC_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### HFOV parameters

#### `VThf` and `VThf_set`
(HFOV-VG)

In [ ]:
if {'VThf', 'VThf_set'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['VThf'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb[start:end]['VThf_set'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 1, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 1, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 6)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VThf', 'VThf_set'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VThf_VThfset', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VThf', 'VThf_set'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['VThf'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb[start:end]['VThf_set'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 1, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 1, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 6)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VThf', 'VThf_set'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VThf_VThfset_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VThf', 'VThf_set'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['VThf'].plot(ax = ax, color = 'blue', x_compat = True,)
    slow_measurements_comb[start:end]['VThf_set'].plot(ax = ax, color = 'black', x_compat = True )
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 1, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 1, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 6)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VThf', 'VThf_set'], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VThf_VThfset_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

#### `VThf`
(HFOV without VG)

In [ ]:
if {'VThf',}.issubset(slow_measurements_comb.columns) and 'VThf_set' not in slow_measurements_comb.columns:

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['VThf'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 1, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 1, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 6)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VThf',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VThf', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VThf',}.issubset(slow_measurements_comb.columns) and 'VThf_set' not in slow_measurements_comb.columns:
    
    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['VThf'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 1, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 1, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 6)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VThf',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VThf_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'VThf',}.issubset(slow_measurements_comb.columns) and 'VThf_set' not in slow_measurements_comb.columns:

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['VThf'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 1, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 1, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 6)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['VThf',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'VThf_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

#### `MAP` and `MAPhf_set`

In [ ]:
if {'MAP', 'MAPhf_set'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['MAP'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['MAPhf_set'].plot(ax = ax, color = 'black', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 5, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 5, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 30)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MAPhf', 'MAPhf_set',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MAPhf_MAPhfset', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MAP', 'MAPhf_set'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['MAP'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['MAPhf_set'].plot(ax = ax, color = 'black', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 5, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 5, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 30)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MAPhf', 'MAPhf_set',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MAPhf_MAPhfset_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'MAP', 'MAPhf_set'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['MAP'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['MAPhf_set'].plot(ax = ax, color = 'black', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 5, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 5, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 30)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['MAPhf', 'MAPhf_set',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'MAPhf_MAPhfset_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

#### `Amplitude` and `Amplhf_max`
(HFOV-VG)

In [ ]:
if {'Amplitude', 'Amplhf_max'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['Amplitude'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['Amplhf_max'].plot(ax = ax, color = 'black', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 10, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 10, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 60)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Amplitude', 'Amplhf_max',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Ampl_Amplhfmax', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'Amplitude', 'Amplhf_max'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['Amplitude'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['Amplhf_max'].plot(ax = ax, color = 'black', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 10, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 10, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 60)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Amplitude', 'Amplhf_max',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Ampl_Amplhfmax_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'Amplitude', 'Amplhf_max'}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['Amplitude'].plot(ax = ax, color = 'red', x_compat = True,)
    slow_measurements_comb[start:end]['Amplhf_max'].plot(ax = ax, color = 'black', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 10, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 10, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 60)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Amplitude', 'Amplhf_max',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Ampl_Amplhfmax_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

#### `Amplitude`
(HFOV without VG)

In [ ]:
if {'Amplitude',}.issubset(slow_measurements_comb.columns) and \
    'Amplhf_max' not in slow_measurements_comb.columns:

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['Amplitude'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 10, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 10, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 60)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Amplitude', 'Amplhf_max',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Ampl', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'Amplitude',}.issubset(slow_measurements_comb.columns) and \
    'Amplhf_max' not in slow_measurements_comb.columns:

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['Amplitude'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 10, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 10, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 60)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Amplitude', 'Amplhf_max',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Ampl_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'Amplitude',}.issubset(slow_measurements_comb.columns) and \
    'Amplhf_max' not in slow_measurements_comb.columns:

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['Amplitude'].plot(ax = ax, color = 'red', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 10, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 10, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 60)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['Amplitude', 'Amplhf_max',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'Ampl_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

#### `DCO2`

In [ ]:
if {'DCO2',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb[start:end]['DCO2'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 15, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 15, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 80)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel(r'mL/kg$^2$sec', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['DCO2',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'DCO2', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'DCO2',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_1min_mean[start:end]['DCO2'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 15, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 15, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 80)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel(r'mL/kg$^2$sec', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['DCO2',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'DCO2_1min_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
if {'DCO2',}.issubset(slow_measurements_comb.columns):

    filetype = 'jpg'
    dpi = 300

    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)

    slow_measurements_comb_3h_rolling_mean[start:end]['DCO2'].plot(ax = ax, color = 'blue', x_compat = True,)
    
    if segment_1_start and segment_1_end:
    
        ax.axvline(pd.to_datetime(segment_1_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_1_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_1_start) + pd.to_timedelta('15M'), 15, '1', fontsize = 14)
    
    if segment_2_start and segment_2_end:
    
        ax.axvline(pd.to_datetime(segment_2_start), color='black', linestyle='dashed', lw=2)
        ax.axvline(pd.to_datetime(segment_2_end), color='black', linestyle='dashed', lw=2)
        ax.text(pd.to_datetime(segment_2_start) + pd.to_timedelta('15M'), 15, '2', fontsize = 14)

    ax.set_ylim(bottom=0)
    #ax.set_ylim(0, 80)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel(r'mL/kg$^2$sec', size = 12, color = 'black')
    ax.set_title('',  size = 12, color = 'black')
    ax.legend(['DCO2',], ncol = 2, fontsize = 12, loc = 'best')
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
    majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
        
    fig.savefig(os.path.join(DATA_DUMP, recording, '%s%s%s.%s' % (recording, tag, 'DCO2_3h_rolling_mean', 
        filetype)), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

### pCO2

In [ ]:
# Limit pCO2 data to the time of the recordings

dt_format = '%Y-%m-%d %H:%M:%S'

pCO2s_filtered = pCO2s[slow_measurements_comb.index[0].strftime(dt_format) :
                       slow_measurements_comb.index[-1].strftime(dt_format)]   

In [ ]:
filetype = 'jpg'
dpi = 300

fig = plt.figure()
fig.set_size_inches(12, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)
a = pCO2s_filtered
     
cap = a[a['Blood specimen type, POC'] == 'Capill...']
art = a[a['Blood specimen type, POC'] == 'Arteri...']
    
if not cap.empty:
    cap['pCO2, POC'].plot(ax = ax, color = 'black', alpha = 0.7, marker = 'o', markerfacecolor = 'green',
                             label = 'capillary CO2', x_compat = True)
    
if not art.empty:
    art['pCO2, POC'].plot(ax = ax, color = 'black', alpha = 0.7, marker = 'o', markerfacecolor = 'red',
                             label = 'arterial CO2', x_compat = True)

majorFmt = dates.DateFormatter('%d/%m\n%H:%M')  
ax.xaxis.set_major_formatter(majorFmt)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')    
    
ax.set_xlabel('Time (hours)', size = 12, color = 'black')
ax.set_ylabel('pCO2 (kPa)', size = 12, color = 'black')
ax.set_title('%s' % recording,  size = 12, color = 'black')
ax.legend(fontsize = 12)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax.set_xlim(slow_measurements_comb.index[0], slow_measurements_comb.index[-1] )
ax.set_ylim(0, a['pCO2, POC'].max() + 2)

fig.savefig('%s/%s/%s_%s.%s' % (DATA_DUMP, recording, recording, 'pCO2', filetype),
        dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

## 12. Produce Figure for the paper

##### Figure 3 color

In [ ]:
dpi = 300
filetype = 'jpg'

fig, ax = plt.subplots(3, 1, figsize = [12, 12])
fig.subplots_adjust(left=0.1, bottom=None, right=0.9, top=None, hspace=0.3, wspace=0.2)

# Figure 3A

slow_measurements_comb['RR'].plot(ax = ax[0], color = 'blue', x_compat = True )
slow_measurements_comb['RRmand'].plot(ax = ax[0], color = 'red', x_compat = True )
slow_measurements_comb['RRspon'].plot(ax = ax[0], color = 'green', x_compat = True )
slow_measurements_comb['RRset'].plot(ax = ax[0], color = 'black', linewidth = 2, x_compat = True)
    
ax[0].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].text(pd.to_datetime('2015-10-19 22:15:00'), 105, '1', fontsize = 14)
    
ax[0].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].text(pd.to_datetime('2015-10-20 17:15:00'), 105, '2', fontsize = 14)

ax[0].set_xlabel('Time (hours)', size = 12, color = 'black')
ax[0].set_ylabel('Inflations/min', size = 12, color = 'black')
ax[0].set_title('',  size = 12, color = 'black')
ax[0].legend(['RR', 'RRmand', 'RRspon', 'RRset'], ncol = 2, fontsize = 12)
ax[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
majorFmt = dates.DateFormatter('%H:%M')  
ax[0].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
   

# Figure 3B

slow_measurements_comb['PIP'].plot(ax = ax[1], color = 'red', x_compat = True,)
slow_measurements_comb['MAP'].plot(ax = ax[1], color = 'blue', x_compat = True )
slow_measurements_comb['PEEP'].plot(ax = ax[1], color = 'green', x_compat = True )
slow_measurements_comb['Pmax'].plot(ax = ax[1], color = 'black', linewidth = 2, x_compat = True)

ax[1].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].text(pd.to_datetime('2015-10-19 22:15:00'), 31, '1', fontsize = 14)
 
ax[1].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].text(pd.to_datetime('2015-10-20 17:15:00'), 31, '2', fontsize = 14)    
    
ax[1].set_xlabel('', size = 12, color = 'black')
ax[1].set_ylabel('mbar', size = 12, color = 'black')
ax[1].set_title('',  size = 12, color = 'black')
ax[1].legend(['PIP', 'MAP', 'PEEP', 'Pmax'], ncol = 2, fontsize = 12)
ax[1].set_ylim(0, 40)
ax[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
majorFmt = dates.DateFormatter('%H:%M')  
ax[1].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')


# Figure 3C

slow_measurements_comb['VTmand'].plot(ax = ax[2], color = 'blue', x_compat = True)
slow_measurements_comb['VTset'].plot(ax = ax[2], color = 'black', linewidth = 2, x_compat = True)

ax[2].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].text(pd.to_datetime('2015-10-19 22:15:00'), 13, '1', fontsize = 14)
    
ax[2].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].text(pd.to_datetime('2015-10-20 17:15:00'), 13, '2', fontsize = 14)
    
ax[2].set_xlabel('Time (hours)', size = 12, color = 'black')
ax[2].set_ylabel('mL/kg', size = 12, color = 'black')
ax[2].set_ylim(0, 16)
ax[2].set_title('',  size = 12, color = 'black')
ax[2].legend(['VTmand', 'VTset'], fontsize = 12)
ax[2].grid(True, linestyle='-', linewidth=0.5, color = 'gray')
        
majorFmt = dates.DateFormatter('%H:%M')  
ax[2].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[2].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')

fig.text(0.04, 0.94, 'A', fontsize = 16); 
fig.text(0.04, 0.67, 'B', fontsize = 16)
fig.text(0.04, 0.40, 'C', fontsize = 16); 

fig.savefig('%s/%s.%s' % (DATA_DUMP, 'Figure_3_col', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1,);

In [ ]:
dpi = 300
filetype = 'jpg'

fig, ax = plt.subplots(3, 1, figsize = [12, 12])
fig.subplots_adjust(left=0.1, bottom=None, right=0.9, top=None, hspace=0.3, wspace=0.2)

# Figure 3A

slow_measurements_comb['RRmand'].plot(ax = ax[0], color = 'red', x_compat = True )
slow_measurements_comb['RRspon'].plot(ax = ax[0], color = 'green', x_compat = True )
slow_measurements_comb['RRset'].plot(ax = ax[0], color = 'black', linewidth = 2, x_compat = True)
    
ax[0].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].text(pd.to_datetime('2015-10-19 22:15:00'), 105, '1', fontsize = 14)
    
ax[0].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].text(pd.to_datetime('2015-10-20 17:15:00'), 105, '2', fontsize = 14)

ax[0].set_xlabel('Time (hours)', size = 12, color = 'black')
ax[0].set_ylabel('Inflations/min', size = 12, color = 'black')
ax[0].set_title('',  size = 12, color = 'black')
ax[0].legend(['RRmand', 'RRspon', 'RRset'], ncol = 3, fontsize = 12)
ax[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
majorFmt = dates.DateFormatter('%H:%M')  
ax[0].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
   

# Figure 3B

slow_measurements_comb['PIP'].plot(ax = ax[1], color = 'red', x_compat = True,)
slow_measurements_comb['Pmax'].plot(ax = ax[1], color = 'black', linewidth = 2, x_compat = True)

ax[1].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].text(pd.to_datetime('2015-10-19 22:15:00'), 31, '1', fontsize = 14)
 
ax[1].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].text(pd.to_datetime('2015-10-20 17:15:00'), 31, '2', fontsize = 14)    
    
ax[1].set_xlabel('', size = 12, color = 'black')
ax[1].set_ylabel('mbar', size = 12, color = 'black')
ax[1].set_title('',  size = 12, color = 'black')
ax[1].legend(['PIP', 'Pmax'], ncol = 2, fontsize = 12)
ax[1].set_ylim(0, 40)
ax[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
majorFmt = dates.DateFormatter('%H:%M')  
ax[1].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')


# Figure 3C

slow_measurements_comb['VTmand'].plot(ax = ax[2], color = 'blue', x_compat = True)
slow_measurements_comb['VTset'].plot(ax = ax[2], color = 'black', linewidth = 2, x_compat = True)

ax[2].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].text(pd.to_datetime('2015-10-19 22:15:00'), 13, '1', fontsize = 14)
    
ax[2].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].text(pd.to_datetime('2015-10-20 17:15:00'), 13, '2', fontsize = 14)
    
ax[2].set_xlabel('Time (hours)', size = 12, color = 'black')
ax[2].set_ylabel('mL/kg', size = 12, color = 'black')
ax[2].set_ylim(0, 16)
ax[2].set_title('',  size = 12, color = 'black')
ax[2].legend(['VTmand', 'VTset'], fontsize = 12)
ax[2].grid(True, linestyle='-', linewidth=0.5, color = 'gray')
        
majorFmt = dates.DateFormatter('%H:%M')  
ax[2].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[2].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')

fig.text(0.04, 0.94, 'A', fontsize = 16); 
fig.text(0.04, 0.67, 'B', fontsize = 16)
fig.text(0.04, 0.40, 'C', fontsize = 16); 

fig.savefig('%s/%s.%s' % (DATA_DUMP, 'Figure_3_col_revision', filetype), dpi = dpi, 
    facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches=None, pad_inches=0.1,);

#### Figure 3 black-and-white

In [ ]:
dpi = 300
filetype = 'jpg'

fig, ax = plt.subplots(3, 1, figsize = [12, 12])
fig.subplots_adjust(left=0.1, bottom=None, right=0.9, top=None, hspace=0.3, wspace=0.2)

# Figure 3A

slow_measurements_comb['RR'].plot(ax = ax[0], color = 'black', alpha = 0.75, x_compat = True )
slow_measurements_comb['RRmand'].plot(ax = ax[0], color = 'black', alpha = 0.5, x_compat = True )
slow_measurements_comb['RRspon'].plot(ax = ax[0], color = 'black', alpha = 0.25, x_compat = True )
slow_measurements_comb['RRset'].plot(ax = ax[0], color = 'black', linewidth = 2, x_compat = True)
    
ax[0].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].text(pd.to_datetime('2015-10-19 22:15:00'), 105, '1', fontsize = 14)
    
ax[0].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].text(pd.to_datetime('2015-10-20 17:15:00'), 105, '2', fontsize = 14)

ax[0].set_xlabel('Time (hours)', size = 12, color = 'black')
ax[0].set_ylabel('Inflations/min', size = 12, color = 'black')
ax[0].set_title('',  size = 12, color = 'black')
ax[0].legend(['RR', 'RRmand', 'RRspon', 'RRset'], ncol = 2, fontsize = 12)
ax[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
majorFmt = dates.DateFormatter('%H:%M')  
ax[0].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
   

# Figure 3B

slow_measurements_comb['PIP'].plot(ax = ax[1], color = 'black', alpha = 0.25, x_compat = True,)
slow_measurements_comb['MAP'].plot(ax = ax[1], color = 'black', alpha = 0.5, x_compat = True )
slow_measurements_comb['PEEP'].plot(ax = ax[1], color = 'black', alpha = 0.75, x_compat = True )
slow_measurements_comb['Pmax'].plot(ax = ax[1], color = 'black', linewidth = 2, x_compat = True)

ax[1].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].text(pd.to_datetime('2015-10-19 22:15:00'), 31, '1', fontsize = 14)
 
ax[1].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].text(pd.to_datetime('2015-10-20 17:15:00'), 31, '2', fontsize = 14)    
    
ax[1].set_xlabel('', size = 12, color = 'black')
ax[1].set_ylabel('mbar', size = 12, color = 'black')
ax[1].set_title('',  size = 12, color = 'black')
ax[1].legend(['PIP', 'MAP', 'PEEP', 'Pmax'], ncol = 2, fontsize = 12)
ax[1].set_ylim(0, 40)
ax[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
majorFmt = dates.DateFormatter('%H:%M')  
ax[1].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')


# Figure 3C

slow_measurements_comb['VTmand'].plot(ax = ax[2], color = 'black', alpha = 0.25, x_compat = True)
slow_measurements_comb['VTset'].plot(ax = ax[2], color = 'black', linewidth = 2, x_compat = True)

ax[2].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].text(pd.to_datetime('2015-10-19 22:15:00'), 13, '1', fontsize = 14)
    
ax[2].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].text(pd.to_datetime('2015-10-20 17:15:00'), 13, '2', fontsize = 14)
    
ax[2].set_xlabel('Time (hours)', size = 12, color = 'black')
ax[2].set_ylabel('mL/kg', size = 12, color = 'black')
ax[2].set_ylim(0, 16)
ax[2].set_title('',  size = 12, color = 'black')
ax[2].legend(['VTmand', 'VTset'], fontsize = 12)
ax[2].grid(True, linestyle='-', linewidth=0.5, color = 'gray')
        
majorFmt = dates.DateFormatter('%H:%M')  
ax[2].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[2].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')

fig.text(0.04, 0.94, 'A', fontsize = 16); 
fig.text(0.04, 0.67, 'B', fontsize = 16)
fig.text(0.04, 0.41, 'C', fontsize = 16); 

fig.savefig('%s/%s.%s' % (DATA_DUMP, 'Figure_3_bw', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1,);

In [ ]:
dpi = 300
filetype = 'jpg'

fig, ax = plt.subplots(3, 1, figsize = [12, 12])
fig.subplots_adjust(left=0.1, bottom=None, right=0.9, top=None, hspace=0.3, wspace=0.2)

# Figure 3A

slow_measurements_comb['RRmand'].plot(ax = ax[0], color = 'black', alpha = 0.6, x_compat = True )
slow_measurements_comb['RRspon'].plot(ax = ax[0], color = 'black', alpha = 0.25, x_compat = True )
slow_measurements_comb['RRset'].plot(ax = ax[0], color = 'black', linewidth = 2, x_compat = True)
    
ax[0].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].text(pd.to_datetime('2015-10-19 22:15:00'), 105, '1', fontsize = 14)
    
ax[0].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[0].text(pd.to_datetime('2015-10-20 17:15:00'), 105, '2', fontsize = 14)

ax[0].set_xlabel('Time (hours)', size = 12, color = 'black')
ax[0].set_ylabel('Inflations/min', size = 12, color = 'black')
ax[0].set_title('',  size = 12, color = 'black')
ax[0].legend(['RRmand', 'RRspon', 'RRset'], ncol = 3, fontsize = 12)
ax[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    
majorFmt = dates.DateFormatter('%H:%M')  
ax[0].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
   

# Figure 3B

slow_measurements_comb['PIP'].plot(ax = ax[1], color = 'black', alpha = 0.6, x_compat = True,)
slow_measurements_comb['Pmax'].plot(ax = ax[1], color = 'black', linewidth = 2, x_compat = True)

ax[1].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].text(pd.to_datetime('2015-10-19 22:15:00'), 31, '1', fontsize = 14)
 
ax[1].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[1].text(pd.to_datetime('2015-10-20 17:15:00'), 31, '2', fontsize = 14)    
    
ax[1].set_xlabel('', size = 12, color = 'black')
ax[1].set_ylabel('mbar', size = 12, color = 'black')
ax[1].set_title('',  size = 12, color = 'black')
ax[1].legend(['PIP', 'Pmax'], ncol = 2, fontsize = 12)
ax[1].set_ylim(0, 40)
ax[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
majorFmt = dates.DateFormatter('%H:%M')  
ax[1].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')


# Figure 3C

slow_measurements_comb['VTmand'].plot(ax = ax[2], color = 'black', alpha = 0.6, x_compat = True)
slow_measurements_comb['VTset'].plot(ax = ax[2], color = 'black', linewidth = 2, x_compat = True)

ax[2].axvline(pd.to_datetime('2015-10-19 22:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].axvline(pd.to_datetime('2015-10-19 23:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].text(pd.to_datetime('2015-10-19 22:15:00'), 13, '1', fontsize = 14)
    
ax[2].axvline(pd.to_datetime('2015-10-20 17:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].axvline(pd.to_datetime('2015-10-20 18:00:00'), color='black', linestyle='dashed', lw=2)
ax[2].text(pd.to_datetime('2015-10-20 17:15:00'), 13, '2', fontsize = 14)
    
ax[2].set_xlabel('Time (hours)', size = 12, color = 'black')
ax[2].set_ylabel('mL/kg', size = 12, color = 'black')
ax[2].set_ylim(0, 16)
ax[2].set_title('',  size = 12, color = 'black')
ax[2].legend(['VTmand', 'VTset'], ncol = 2, fontsize = 12)
ax[2].grid(True, linestyle='-', linewidth=0.5, color = 'gray')
        
majorFmt = dates.DateFormatter('%H:%M')  
ax[2].xaxis.set_major_formatter(majorFmt)
plt.setp(ax[2].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')

fig.text(0.04, 0.94, 'A', fontsize = 16); 
fig.text(0.04, 0.67, 'B', fontsize = 16)
fig.text(0.04, 0.41, 'C', fontsize = 16); 

fig.savefig('%s/%s.%s' % (DATA_DUMP, 'Figure_3_bw_revision', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1,);